In [ ]:
# Install needed packages
!pip install transformers datasets torch accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 1. Load necessary libraries
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import torch

In [ ]:
# 2. Load SPIDER dataset (via Huggingface hub)
dataset = load_dataset("spider")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.51k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/831k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [ ]:
train_data = dataset['train']
val_data = dataset['validation']

In [ ]:
print(f"Train size: {len(train_data)}, Validation size: {len(val_data)}")

Train size: 7000, Validation size: 1034


In [ ]:
# 3. Preprocessing function to add table schema info to the question
def preprocess_function(examples):
    inputs = []
    targets = []
    for question, sql, db_schema in zip(examples["question"], examples["query"], examples["db_id"]):
        prompt = f"Translate to SQL (db: {db_schema}): {question}"
        inputs.append(prompt)
        targets.append(sql)
    return {"input_text": inputs, "target_text": targets}

In [ ]:
# Apply preprocessing
train_data = train_data.map(preprocess_function, batched=True, remove_columns=train_data.column_names)
val_data = val_data.map(preprocess_function, batched=True, remove_columns=val_data.column_names)

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [ ]:
# 4. Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# 5. Tokenize dataset
def tokenize_function(examples):
    model_inputs = tokenizer(examples["input_text"], max_length=512, padding="max_length", truncation=True)
    labels = tokenizer(examples["target_text"], max_length=512, padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [ ]:
# 6. Define Data Collator (for dynamic padding)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from transformers import Seq2SeqTrainingArguments


In [ ]:
# 7. Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results_nl2sql_spider",
    eval_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    logging_steps=200,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=3,
    learning_rate=3e-4,
    save_total_limit=2,
    fp16=True,                      # Enable mixed precision training
    #predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

In [ ]:
# 8. Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-16-cdc590ba0ee5>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# 9. Start Training
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pranayvadla20cm002 (pranayvadla20cm002-keshav-memorial-college-of-law) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
500,0.085900,0.109377


TrainOutput(global_step=657, training_loss=0.1820896444799693, metrics={'train_runtime': 941.5741, 'train_samples_per_second': 22.303, 'train_steps_per_second': 0.698, 'total_flos': 2842177830912000.0, 'train_loss': 0.1820896444799693, 'epoch': 3.0})

In [ ]:
# 10. Save the final model
model.save_pretrained("./final_nl2sql_spider_model")
tokenizer.save_pretrained("./final_nl2sql_spider_model")

('./final_nl2sql_spider_model/tokenizer_config.json',
 './final_nl2sql_spider_model/special_tokens_map.json',
 './final_nl2sql_spider_model/spiece.model',
 './final_nl2sql_spider_model/added_tokens.json')

In [ ]:
# 11. Example Inference Function
def generate_sql(question, db_name="some_db"):
    prompt = f"Translate to SQL (db: {db_name}): {question}"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=512).to(model.device)
    output = model.generate(**inputs, max_length=512)
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
# Example usage
example_question = " Write a SQL query to find the customer who ordered the most keyboards, using pattern matching and aggregate functions."
print("Generated SQL:\n", generate_sql(example_question, db_name="customer_orders"))

Generated SQL:
 SELECT customer_order_code FROM orders AS t1 JOIN customer_orders AS t2 ON t1.customer_id = t2.customer_id JOIN customer_orders AS t3 ON t1.customer_id = t3.customer_id JOIN customer_orders AS t3 ON t2.customer_id = t3.customer_id GROUP BY t1.customer_id = SELECT t1.customer_code ORDER BY count(*) DESC LIMIT 1
